# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import Reader
from firefly_api.particlegroup import ParticleGroup
import numpy as np
import pandas as pd

import sys
import json
import requests
from IPython.display import IFrame

### Start the server

In [2]:
%%bash --bg --proc bg_proc
python /Users/ageller/Visualizations/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [13]:
url = "http://localhost:5000/combined"
IFrame(url, width=1000, height=500)

### Create a new "reader", that will accept particles and output a properly formatted dict

In [8]:
def simpleReader(name, coords, color, sizeMult):
    #create the reader
    my_reader = Reader(
        ## the name of the sub-directory that will contain your JSON files,
        ##  if you are not running python from /path/to/Firefly/data it should be the absolute path
        JSONdir = '/foo/', #this is silly, but required for now
        ## options object, if you have one you can pass it, None will generate default options, see
        ##  below for options arguments/capabilities
        options = None, 
        ## whether a startup.json file should be written, defaults to 'append'
        doValidate = False
    )

    #define a particle group
    my_parts = ParticleGroup(
        name,
        coords,
        ## below here I pass a few option_kwargs to set the size and color of the points
        sizeMult=sizeMult, 
        color = color, 
    )

    ## tell the reader to keep track of this particle group
    my_reader.addParticleGroup(my_parts)

    ## Store this info in dicts with the correct format for Firefly
    outDict = {"parts":{}, "options":None}
    for pg in my_reader.particleGroups:
        outDict['parts'][pg.UIname] = pg.outputToDict()
    outDict['options'] = my_reader.options.outputToDict()
    
    return outDict


### Create some data, and get the Firefly dict

In [9]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T
print(coords.shape)

## make the particles a bit easier to see since there's so few of them
sizeMult = 5

## colors should be RGBA lists
color = [1,1,0,1]

## choose some name
name = 'part1'

outDict = simpleReader(name, coords, color, sizeMult)
#print(outDict)
print(outDict['parts'][name]['Coordinates'].shape)

(8000, 3)
(8000, 3)


## Send this data to the Flask app

*This sends the data.  (Then look back above to see it.)*

In [14]:
s = pd.Series(outDict).to_json()
requests.post("http://localhost:5000/data_input", json=s)

<Response [200]>

*To kill the Firefly server.*

In [ ]:
# uncomment the next line
#! ps aux | grep Firefly | awk '{print $2}' | xargs kill
